In [539]:
from IPython.display import display, Math, Latex
from itertools import product, combinations
import copy
    

class monomial:
    #just initialization
    def __init__(self, fixed_part, running_indices):
        self.fixed_part = fixed_part.copy()
        self.running_indices = running_indices.copy()
        self.list_of_indices = {elem[0] : elem[1] for elem in running_indices.keys()}
    #gathering a latex style string to represent monomial    
    def __str__(self):
        result = ""
        f_helper = []
        r_helper = []
        for i in self.fixed_part.keys():
            f_helper.append(i)
        for i in self.running_indices.keys():
            r_helper.append(i)
        f_helper.sort()
        r_helper.sort(key = lambda x: (x[0], x[1]))
        for i in f_helper:
            result += "x_" + str(i)
            if self.fixed_part[i] > 1:
                result += "^{" + str(self.fixed_part[i]) + "}"
        for i in r_helper:
            result += "x_{" + i[0] + "+" + str(i[1]) + "}"
            if self.running_indices[i] > 1:
                result += "^{" + str(self.running_indices[i]) + "}"
        result = result.replace("+0", "")
        if len(result) == 0:
            result = "1"
        return result
    
    #overriding addition
    def __add__(self, other):
        if type(self) == type(other):
            if self == other:
                return polynomial({self : 2})
            else:
                return polynomial({self : 1, other : 1})
        elif type(other) == polynomial:
            result = other.monomial_dict
            if self in result.monomial_dict.keys():
                result[self] += 1
            else:
                result[self] = 1
            return polynomial(result)
            
    #overriding subtraction
    def __sub__(self, other):
        if type(self) == type(other):
            if self == other:
                return polynomial({})
            else:
                return polynomial({self : 1, other : -1})
    #overriding multiplication
    def __mul__(self, other):
        if type(self) == type(other):
            result_fix = self.fixed_part.copy()
            result_run = self.running_indices.copy()
            for elem in other.fixed_part.keys():
                if elem in result_fix.keys():
                    result_fix[elem] += other.fixed_part[elem]
                else:
                    result_fix[elem] = other.fixed_part[elem]
            for elem in other.running_indices.keys():
                if elem in result_run.keys():
                    result_run[elem] += other.running_indices[elem]
                else:
                    result_run[elem] = other.running_indices[elem]
            return monomial(result_fix, result_run)
        elif type(other) == polynomial:
            result = polynomial({self * elem : other.monomial_dict[elem] for elem in other.monomial_dict.keys()})
            return result
    def __rmul__(self, other):
        return self.__mul__(other)
    #equality check
    def __eq__(self, other):
        return ((self.fixed_part == other.fixed_part) and (self.running_indices == other.running_indices))
    #makes hash out from string representation in order to use monomials as dict keys
    def __hash__(self):
        return hash(self.__str__())
    #changes every index_to_change into what_to_insert (argument 
    #must be of form tuple of elements ("index_to_change", what_to_insert = ("index" , number added to "index")))
    #this method changes self
    def self_substitution(self, insertion_tuple):
        for elem in insertion_tuple:
            index_to_change = elem[0]
            what_to_insert = elem[1]
            #print(elem)
            #print(index_to_change, what_to_insert)
            if index_to_change in self.list_of_indices.keys():
                if what_to_insert[0] != "":
                    helper = self.running_indices.copy()
                    for elem1 in helper.keys():
                        if index_to_change == elem1[0]:
                            b = what_to_insert[1] + elem1[1]
                            deg = self.running_indices[elem1]
                            self.running_indices.pop(elem1)
                            self.list_of_indices.pop(index_to_change)
                            if (what_to_insert[0], b) in self.running_indices.keys():
                                self.running_indices[(what_to_insert[0], b)] += deg
                            else:
                                self.running_indices[(what_to_insert[0], b)] = deg
                                self.list_of_indices[what_to_insert[0]] = b
        for elem in insertion_tuple:
            index_to_change = elem[0]
            what_to_insert = elem[1]
            #print(elem)
            #print(index_to_change, what_to_insert)
            if index_to_change in self.list_of_indices.keys():
                if what_to_insert[0] == "":
                    helper = self.running_indices.copy()
                    for elem1 in helper.keys():
                        if index_to_change == elem1[0]:
                            b = what_to_insert[1] + elem1[1]
                            deg = self.running_indices[elem1]
                            self.running_indices.pop(elem1)
                            if b in self.fixed_part.keys():
                                self.fixed_part[b] += deg
                            else:
                                self.fixed_part[b] = deg
        
                #self.list_of_indices.pop(index_to_change)
                #self.list_of_indices[what_to_insert[0]] = b
                
    #does the same as self_substitution, without changing self, but creating a copy        
    def copy_substitution(self, insertion_tuple):
        result = monomial(self.fixed_part, self.running_indices)
        result.self_substitution(insertion_tuple)
        return result
    
    #if return_gcd is true returns a three element tuple (gcd, self freed from gcd, other freed from gcd)
    #otherwise returns only freed self and other
    def free_from_gcd(self, other, return_gcd):
        self_copy = monomial(self.fixed_part, self.running_indices)
        other_copy = monomial(other.fixed_part, other.running_indices)
        sure_fixed_part = {}
        sure_running_indices = {}
        helper = self_copy.running_indices.copy()
        for elem in helper.keys():
            if elem in other_copy.running_indices.keys():
                if self_copy.running_indices[elem] == other_copy.running_indices[elem]:
                    sure_running_indices[elem] = self_copy.running_indices[elem]
                    self_copy.running_indices.pop(elem)
                    other_copy.running_indices.pop(elem)
                elif other_copy.running_indices[elem] < self_copy.running_indices[elem]:
                    sure_running_indices[elem] = other_copy.running_indices[elem]
                    self_copy.running_indices[elem] -= other_copy.running_indices[elem]
                    other_copy.running_indices.pop(elem)
                else:
                    sure_running_indices[elem] = self_copy.running_indices[elem]
                    other_copy.running_indices[elem] -= self_copy.running_indices[elem]
                    self_copy.running_indices.pop(elem)
        helper = self_copy.fixed_part.copy()
        for elem in helper.keys():
            if elem in other_copy.fixed_part.keys():
                if self_copy.fixed_part[elem] == other_copy.fixed_part[elem]:
                    sure_fixed_part[elem] = self_copy.fixed_part[elem]
                    self_copy.fixed_part.pop(elem)
                    other_copy.fixed_part.pop(elem)
                elif other_copy.fixed_part[elem] < self_copy.fixed_part[elem]:
                    sure_fixed_part[elem] = other_copy.fixed_part[elem]
                    self_copy.fixed_part[elem] -= other_copy.fixed_part[elem]
                    other_copy.fixed_part.pop(elem)
                else:
                    sure_fixed_part[elem] = self_copy.fixed_part[elem]
                    other_copy.fixed_part[elem] -= self_copy.fixed_part[elem]
                    self_copy.running_indices.pop(elem)
        if return_gcd:
            return (monomial(sure_fixed_part, sure_running_indices), self_copy, other_copy)
        else:
            return (self_copy, other_copy)
    
    #finds any possible substitution leading to self and other having common variables
    def possible_intersections(self, other):
        possible_running_indices = {}
        self_copy, other_copy = self.free_from_gcd(other, return_gcd = False)
        #display(Math(self_copy.__str__()))
        #display(Math(other_copy.__str__()))
        for elem in self_copy.running_indices.keys():
            listik = []
            for elem1 in other_copy.fixed_part.keys():
                listik.append(("", elem1 - elem[1]))
                #print(elem[0], "should be equal to ", elem1 - elem[1])
            for elem2 in other_copy.running_indices.keys():
                if elem2[0] != elem[0]:
                    listik.append((elem2[0], elem2[1] - elem[1]))
                    #print(elem[0], "should be equal to ", elem2[0], "+", elem2[1] - elem[1])
            if len(listik) > 0:
                possible_running_indices[elem[0]] = listik
        for elem in other_copy.running_indices.keys():
            listik = []
            for elem1 in self_copy.fixed_part.keys():
                listik.append(("", elem1 - elem[1]))
            if elem[0] in possible_running_indices.keys():
                possible_running_indices[elem[0]].extend(listik)
            elif len(listik) > 0:
                possible_running_indices[elem[0]] = listik
        result = [list(set([(key, value)  for value in possible_running_indices[key]]))
                  for key in possible_running_indices.keys()]
        return result
    
    #just pretty latex printing function
    def print(self):
        display(Math(self.__str__()))
            

class polynomial:
    
    #just initialization
    def __init__(self, monomial_dict):
        self.monomial_dict = monomial_dict.copy()
        #self.monomial_dict = copy.deepcopy(monomial_dict)
        for elem in self.monomial_dict.copy().keys():
            if self.monomial_dict[elem] == 0:
                self.monomial_dict.pop(elem)
                    
                
    #overriding addition for polynomials            
    def __add__(self, other):
        if type(self) == type(other):
            result_dict = self.monomial_dict.copy()
            #result_dict = copy.deepcopy(self.monomial_dict)
            for elem in other.monomial_dict.keys():
                if elem in result_dict.keys():
                    result_dict[elem] += other.monomial_dict[elem]
                else:
                    result_dict[elem] = other.monomial_dict[elem]
            return polynomial(result_dict)
        elif type(other) == monomial:
            return other * self
        
    #overriding subtraction for polynomials
    def __sub__(self, other):
        if type(self) == type(other):
            result_dict = self.monomial_dict.copy()
            #result_dict = copy.deepcopy(self.monomial_dict)
            for elem in other.monomial_dict.keys():
                if elem in list(result_dict.keys()):
                    result_dict[elem] -= other.monomial_dict[elem]
                else:
                    result_dict[elem] = -other.monomial_dict[elem]
            return polynomial(result_dict)
    #overriding multiplication for polynomials
    def __mul__(self, other):
        if type(self) == type(other):
            result = polynomial({})
            for elem in self.monomial_dict.keys():
                helper = elem * other
                for elem1 in helper.monomial_dict.keys():
                    helper.monomial_dict[elem1] *= self.monomial_dict[elem]
                result += helper
            return result
        elif type(other) == monomial:
            return other * self
    #gathering a latex style string in order to represent a polynomial
    def __str__(self):
        result = ""
        for elem in self.monomial_dict.keys():
            if self.monomial_dict[elem] != 1 and self.monomial_dict[elem] != -1:
                result += "+" + str(self.monomial_dict[elem]) + elem.__str__()
            elif self.monomial_dict[elem] == -1:
                result += "-" + elem.__str__()
            else:
                result += "+" + elem.__str__()
        result = result.replace("+-", "-")
        result = result.replace("++", "+")
        if len(result) == 0:
            result = "0"
        if result[0] == "+":
            result = result[1 : ]
        
        return result
    #performs a substitution same to the self_substitution for monomials
    def self_substitution(self, insertion_tuple):
        for elem in self.monomial_dict.keys():
            elem.substitution(insertion_tuple)
    #performs a substitution same to the copy_substitution for monomials
    def copy_substitution(self, insertion_tuple):
        result = {elem.copy_substitution(insertion_tuple) : self.monomial_dict[elem] 
                  for elem in self.monomial_dict.keys()}
        return polynomial(result)
    #pretty latex printing function
    def print(self):
        display(Math(self.__str__()))
        
def list_of_substitutions(l):
    helper = list(product(*l))
    total = []
    for i in helper:
        helper1 = []
        for j in range(len(i) + 1):
            helper1.extend(list(combinations(i, j)))
        total.extend(helper1)
    total = list(set(total))
    return total
#prints a substitution of running indices (sub) which should be of form tuple of (index_to_change, what_to_insert)
def print_sub(sub):
    string = ""
    for elem in sub:
        string += elem[0] + r" \rightarrow " + elem[1][0] + " +" + str(elem[1][1]) + r", \ "
    string = string.replace("+-", "-")
    string = string.replace("+0", "")
    display(Math(string))
    #computes S_polynomial considering all running indices being different enough so that leading monomials have no 
    #common variables other than obvious ones
def S_polynomial(polynomial_1, polynomial_2, leading_monomial_1, leading_monomial_2):
    leader_1, leader_2 = leading_monomial_1.free_from_gcd(leading_monomial_2, return_gcd = False)
    return leader_2 * polynomial_1 - leader_1 * polynomial_2
#computes every_possible S_polynomial considering substitutions which lead to leading monomials having common variables
#other than obvious ones
def every_possible_S_polynomial(polynomial_1, polynomial_2, leading_monomial_1, leading_monomial_2):
    if leading_monomial_1 in polynomial_1.monomial_dict.keys() and leading_monomial_2 in polynomial_2.monomial_dict.keys():
        helper = leading_monomial_1.possible_intersections(leading_monomial_2)
        total = list_of_substitutions(helper)
        S_polynomials = []
        total.sort(key = len)
        for elem in total:
            #print("substitution:")
            f1 = polynomial_1.copy_substitution(elem)
            f2 = polynomial_2.copy_substitution(elem)
            l1 = leading_monomial_1.copy_substitution(elem)
            l2 = leading_monomial_2.copy_substitution(elem)
            #print_sub(elem)
            #print("leads to S_polynomial:")
            f = S_polynomial(f1, f2, l1, l2)
            S_polynomials.append(f)
            #f.print()
            #print("_" * 50)
        return (total, S_polynomials)
    

In [324]:
m = monomial({0 : 1, 9 : 3}, {("k_1", 1) : 1, ("k_2", 1) : 1})

In [325]:
m.print()

<IPython.core.display.Math object>

In [326]:
m10 = m.copy_substitution((("k_1", ("p", 1)), ("p", ("", 1))))

<IPython.core.display.Math object>

('k_1', ('p', 1))
dict_keys(['k_1', 'k_2'])


<IPython.core.display.Math object>

('p', ('', 1))
dict_keys(['p', 'k_2'])


In [327]:
m10.print()

<IPython.core.display.Math object>

In [355]:
m1 = monomial({0 : 1}, {("k", 1) : 1})
m2 = monomial({1 : 1}, {("k", 0) : 1})
f1 = m1 - m2

In [356]:
f1.print()

<IPython.core.display.Math object>

In [357]:
f2 = f1.copy_substitution((("k", ("l", 0)),))

In [358]:
f2.print()

<IPython.core.display.Math object>

In [359]:
m3 = m1.copy_substitution((("k", ("l", 0)),))

In [360]:
m3.print()

<IPython.core.display.Math object>

In [361]:
p = every_possible_S_polynomial(f1, f2, m1, m3)

substitution:


<IPython.core.display.Math object>

leads to S_polynomial:


<IPython.core.display.Math object>

__________________________________________________
substitution:


<IPython.core.display.Math object>

leads to S_polynomial:


<IPython.core.display.Math object>

__________________________________________________


In [530]:
m1 = monomial({0 : 1}, {("k_1", 1) : 1, ("k_2", 5) : 1})
m2 = monomial({1 : 1}, {("k_1", 0) : 1, ("k_2", 4) : 1})

In [531]:
m1.print()
m2.print()

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [532]:
f1 = m1 - m2

In [533]:
f1.print()

<IPython.core.display.Math object>

In [534]:
f2 = f1.copy_substitution((("k_1", ("l_1", 0)), ("k_2", ("l_2", 0))))
m3 = m1.copy_substitution((("k_1", ("l_1", 0)), ("k_2", ("l_2", 0))))

In [535]:
f2.print()
m3.print()

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [536]:
f = every_possible_S_polynomial(f1, f2, m1, m3)

In [538]:
for i in range(len(f[0])):
    print("substitution:")
    print_sub(f[0][i])
    print("leads to S_polynomial:")
    f[1][i].print()
    print("_" * 50)

substitution:


<IPython.core.display.Math object>

leads to S_polynomial:


<IPython.core.display.Math object>

__________________________________________________
substitution:


...

__________________________________________________
substitution:


<IPython.core.display.Math object>

leads to S_polynomial:


<IPython.core.display.Math object>

__________________________________________________
